In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [19]:
# Get the data from tsm and qcom with inteval of 30 minutes

tsm = yf.Ticker("TSM")
qcom = yf.Ticker("QCOM")

tsm_data = tsm.history(period="30d", interval="60m")
qcom_data = qcom.history(period="30d", interval="60m")

In [20]:
# plot both tickers on go figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close'], name='QCOM'))

fig.show()

In [21]:
print(tsm_data)

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-03-15 09:30:00-04:00  135.440002  137.289993  134.630005  136.625107   
2024-03-15 10:30:00-04:00  136.630005  137.699997  136.199997  136.565002   
2024-03-15 11:30:00-04:00  136.565002  136.809998  136.103195  136.800003   
2024-03-15 12:30:00-04:00  136.794998  137.389999  136.490005  136.979996   
2024-03-15 13:30:00-04:00  136.964996  137.570007  136.964996  137.250000   
...                               ...         ...         ...         ...   
2024-04-26 11:30:00-04:00  137.919998  138.130005  137.539993  137.589996   
2024-04-26 12:30:00-04:00  137.600006  138.485001  137.494995  138.310104   
2024-04-26 13:30:00-04:00  138.330002  138.429993  137.750000  137.986893   
2024-04-26 14:30:00-04:00  138.009995  138.139999  137.699997  138.089996   
2024-04-26 15:30:00-04:00  138.119995  138.539993  137.779999  138.300003   

In [22]:
# plot eith candle stick

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

# plot moving average of 20 ticks and 80 ticks

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()

In [23]:
# count the number of times the 5 ticks MA crosses the 20 ticks MA

tsm_data['5T_MA'] = tsm_data['Close'].rolling(window=5).mean()
tsm_data['20T_MA'] = tsm_data['Close'].rolling(window=20).mean()

tsm_data['signal'] = np.where(tsm_data['5T_MA'] > tsm_data['20T_MA'], 1, 0)
tsm_data['signal'] = tsm_data['signal'].diff()


print(tsm_data['signal'].value_counts())

# plot the buy and sell signals

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))

fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

signal
 0.0    202
 1.0      4
-1.0      3
Name: count, dtype: int64


In [24]:
# calculate profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)

tsm_data['buy_price'] = tsm_data['buy_price'].ffill()
tsm_data['sell_price'] = tsm_data['sell_price'].ffill()

tsm_data['PnL'] = tsm_data['sell_price'] - tsm_data['buy_price']

print(tsm_data['PnL'].sum())

# plot the profit and loss

fig = go.Figure(data=[go.Candlestick(x=tsm_data.index,
                open=tsm_data['Open'],
                high=tsm_data['High'],
                low=tsm_data['Low'],
                close=tsm_data['Close'])])

fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'].rolling(window=20).mean(), name='20 Ticks MA'))

buy_signals = tsm_data[tsm_data['signal'] == 1]
sell_signals = tsm_data[tsm_data['signal'] == -1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

fig.show()

150.775390625


In [25]:
ratio_0 = (tsm_data['Close'].iloc[0]) / (qcom_data['Close'].iloc[0])
ratio_0

0.8149429549406286

In [26]:
qcom_data['Close_NORM'] = qcom_data['Close'].multiply(ratio_0).round(6)

In [27]:
tsm_qcom_dif = tsm_data['Close'] - qcom_data['Close_NORM']
tsm_qcom_dif

Datetime
2024-03-15 09:30:00-04:00   -1.884766e-07
2024-03-15 10:30:00-04:00    1.316244e-02
2024-03-15 11:30:00-04:00    2.481631e-01
2024-03-15 12:30:00-04:00    9.823117e-01
2024-03-15 13:30:00-04:00    9.344880e-01
                                 ...     
2024-04-26 11:30:00-04:00    3.157001e+00
2024-04-26 12:30:00-04:00    2.691378e+00
2024-04-26 13:30:00-04:00    2.828599e+00
2024-04-26 14:30:00-04:00    2.638322e+00
2024-04-26 15:30:00-04:00    3.296550e+00
Length: 210, dtype: float64

In [28]:
# Plot the difference between the two tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))

fig.show()

In [29]:
# Plot the two normalized tickers

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_data.index, y=tsm_data['Close'], name='TSM'))
fig.add_trace(go.Scatter(x=qcom_data.index, y=qcom_data['Close_NORM'], name='QCOM'))

fig.show()

In [30]:
# Plot the difference and its moving average

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif, name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=5).mean(), name='5 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=20).mean(), name='20 Ticks MA'))

fig.show()


In [31]:
tsm_qcom_dif = tsm_qcom_dif.to_frame()
tsm_qcom_dif.columns = ['Close']

In [32]:
tsm_qcom_dif['10T_MA'] = tsm_qcom_dif['Close'].rolling(window=10).mean()
tsm_qcom_dif['60T_MA'] = tsm_qcom_dif['Close'].rolling(window=60).mean()

tsm_qcom_dif['signal'] = np.where(tsm_qcom_dif['10T_MA'] > tsm_qcom_dif['60T_MA'], 1, 0)
tsm_qcom_dif['signal'] = tsm_qcom_dif['signal'].diff()

print(tsm_qcom_dif['signal'].value_counts())

# Plot the signals

fig = go.Figure()
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif['Close'], name='TSM-QCOM'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=10).mean()['Close'], name='10 Ticks MA'))
fig.add_trace(go.Scatter(x=tsm_qcom_dif.index, y=tsm_qcom_dif.rolling(window=50).mean()['Close'], name='50 Ticks MA'))

buy_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == -1]
sell_signals = tsm_qcom_dif[tsm_qcom_dif['signal'] == 1]

fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(color='red', size=10), name='Sell Signal'))

 # -1 buy || 1 sell

tsm_data['signal'] = -tsm_qcom_dif['signal']
qcom_data['signal'] = tsm_qcom_dif['signal'] # opposite signal

fig.show()

# plot the profit and loss

tsm_data['buy_price'] = np.where(tsm_data['signal'] == -1, tsm_data['Close'], np.nan)
tsm_data['sell_price'] = np.where(tsm_data['signal'] == 1, tsm_data['Close'], np.nan)

qcom_data['buy_price'] = np.where(qcom_data['signal'] == -1, qcom_data['Close_NORM'], np.nan)
qcom_data['sell_price'] = np.where(qcom_data['signal'] == 1, qcom_data['Close_NORM'], np.nan)

tsm_profit = tsm_data['sell_price'].sum() - tsm_data['buy_price'].sum()
qcom_profit = qcom_data['sell_price'].sum() - qcom_data['buy_price'].sum()

print('TSMC Profit => ', tsm_profit)
print('QCOM Profit => ', qcom_profit)

pair_tsm_qcom_profit = tsm_profit + qcom_profit

print('Pair profit => ', pair_tsm_qcom_profit)


signal
 0.0    200
 1.0      5
-1.0      4
Name: count, dtype: int64


TSMC Profit =>  -137.79000854492188
QCOM Profit =>  128.96878800000002
Pair profit =>  -8.821220544921857


In [33]:
#Consider TSM as the first and QCOM as the second ticker.
#When there is a buy signal for TSM, we buy TSM and sell QCOM.
#When there is a sell signal for TSM, we sell TSM and buy QCOM.

In [34]:
#Copy signal column from difference dataframe to TSM dataframe
tsm_data['signal'] = tsm_qcom_dif['signal']
qcom_data['signal'] = -tsm_qcom_dif['signal']